<a href="https://colab.research.google.com/github/cpesantez/TFM-OpenNebula-CarlaPesantez/blob/main/Copia_de_TFMlocal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#*COMPLEMENTO TÉCNICO PARA OPEN NEBULA-TFM CARLA PESANTEZ*
Version adaptada a Entorno Local con Minione



Este script esta pensado para ejecutarse sobre un entorno OpenNebula desplegado con MiniONE en una VM local (Ubuntu + VirtualBox)
- Entorno principal actual: "local_minione"
- El diseno permite anadir en el futuro un entorno "remote_nube" cambiando solo la configuracion, sin tocar la logica de los modulos.

# MODULOS:
1. Gestion del entorno y parametros de configuracion
2. Autenticación contra la API XML-RPC de OpenNebula
3. Despliegue de máquinas virtuales desde una plantilla
4. Simulacion de Configuración de red  
5. Registro de eventos del complemento
6. Flujo principal de ejemplo (una VM tipo laboratorio CTF)




El presente codigo corresponde a la version validada en entorno local (MiniONE sobre Ubuntu 22.04 y VirtualBox)

In [ ]:
# Carga de librerías

import os
import datetime
import xmlrpc.client # Para hablar con la API de OpenNebula


# MODULO 1. Configuracion del entorno y parametros de ejecucion

In [ ]:

# SELECCION DEL ENTORNO ACTIVO

ENTORNO_ACTIVO = "local_minione"
# Diccionario de Entornos disponibles
ENTORNOS = {
    "local_minione": {
        "descripcion": "Entorno local basado en OpenNebula MiniONE",
        "ip": "127.0.0.1",
        "puerto_rpc": 2633,
        "usuario": "oneadmin",
        "contrasena": "A64ghqSHXh" #Se reemplaza al ejecutar realmente
    }
    # Futuro entorno remoto podria anadirse aqui sin modificar los modulos:
    # "nube_publica":{....}
}

# Modo Simulacion:
# True = solo imprime acciones y registra logs (No ejecuta API)
# False = llamadas reales a la API de OpenNebula
MODO_SIMULACION = True

def obtener_configuracion(entorno: str):
    """Obtiene los parámetros (IP, puerto y credenciales) del entorno activo."""
    cfg = ENTORNOS.get(entorno)
    if not cfg:
        raise ValueError(f"El entorno '{entorno}' no está definido.")
    endpoint = f"http://{cfg['ip']}:{cfg['puerto_rpc']}/RPC2"
    return cfg["usuario"], cfg["contrasena"], endpoint

In [ ]:
print(f"El valor actual de MODO_SIMULACION es: {MODO_SIMULACION}")

El valor actual de MODO_SIMULACION es: True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#MODULO 2.Autenticación con la API XML-RPC DE OPENNEBULA

* Este módulo conecta la API de OpenNebula.

In [ ]:
def conectar_api(entorno: str = ENTORNO_ACTIVO):
    """
    Establece conexión con el endpoint RPC de OpenNebula.
    Si MODO_SIMULACION está activado, devuelve un cliente simulado.
    """
    usuario, password, endpoint = obtener_configuracion(entorno)

    if MODO_SIMULACION:
        print(f"[SIMULACIÓN] Conexión ficticia a {endpoint} como {usuario}")
        return "sesion_simulada", None

    try:
        session = f"{usuario}:{password}"
        client = xmlrpc.client.ServerProxy(endpoint)

        # Llamada de validación: versión del sistema
        ok, version, error = client.one.system.version(session)
        if not ok:
            raise RuntimeError(error)

        print(f"Conexión establecida correctamente. OpenNebula versión: {version}")
        return session, client

    except Exception as e:
        print("Error en la conexión con OpenNebula:", e)
        return None, None

#MODULO 3.Despliegue de MVs



In [ ]:
def crear_vm(nombre_vm: str, template_id: int, session=None, client=None):
    """
    Instancia una máquina virtual en OpenNebula a partir de una plantilla base.
    """
    if MODO_SIMULACION or client is None or session is None:
        print(f"[SIMULACIÓN] Creación de VM '{nombre_vm}' desde plantilla {template_id}")
        return f"sim-{nombre_vm}"

    try:
        ok, vm_id, error = client.one.template.instantiate(
            session,
            template_id,
            nombre_vm,
            False,   # hold
            "",      # extra_template
            False    # persistent
        )

        if not ok:
            raise RuntimeError(error)

        print(f"VM REAL creada con ID {vm_id}")
        return vm_id

    except Exception as e:
        print("Error creando la VM:", e)
        return None



#MODULO 4.Configuración de Red (Simulada)



In [ ]:
def crear_vm(nombre_vm: str, template_id: int, session=None, client=None):
    """
    Instancia una máquina virtual en OpenNebula a partir de una plantilla base.
    """
    if MODO_SIMULACION or client is None or session is None:
        print(f"[SIMULACIÓN] Creación de VM '{nombre_vm}' desde plantilla {template_id}")
        return f"sim-{nombre_vm}"

    try:
        ok, vm_id, error = client.one.template.instantiate(
            session,
            template_id,
            nombre_vm,
            False,   # hold
            "",      # extra_template
            False    # persistent
        )

        if not ok:
            raise RuntimeError(error)

        print(f"VM REAL creada con ID {vm_id}")
        return vm_id

    except Exception as e:
        print("Error creando la VM:", e)
        return None

#MODULO 5. Registro de Eventos del Complemento
* Este módulo guarda eventos importantes en un archivo de log para trazabilidad y auditoría.

In [ ]:
def registrar_evento(mensaje: str, log_path="logs/eventos_complemento.log"):
    """Registra un mensaje en un archivo de log con marca temporal."""
    os.makedirs(os.path.dirname(log_path), exist_ok=True)
    timestamp = datetime.datetime.now().isoformat(timespec="seconds")
    linea = f"[{timestamp}] {mensaje}"
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(linea + "\n")
    print(f"[LOG] {linea}")

#MODULO 6: Flujo principal del complemento

* Esta función orquesta la ejecución de los módulo anteriores.

In [ ]:
def ejecutar_complemento():
    """
    Flujo principal validado en el TFM:
    - Conexión al entorno local MiniONE
    - Despliegue de una VM de laboratorio
    - Configuración simulada de red
    - Registro de eventos
    """
    print("Inicio de ejecución del complemento...")
    session, client = conectar_api(ENTORNO_ACTIVO)

    if session is None:
        registrar_evento("Error de autenticación. Ejecución abortada.")
        return

    # Plantilla base (ID visible en Sunstone)
    plantilla_id = 1
    nombre_vm = "vm_ctf_local_01"

    vm_id = crear_vm(nombre_vm, plantilla_id, session=session, client=client)
    if vm_id is None:
        registrar_evento("Fallo en la creación de VM.")
        return

    parametros_red = {
        "vnet": "default",
        "descripcion": "Red base del escenario CTF (entorno local)"
    }

    # La función 'configurar_red' no está definida en el cuaderno proporcionado.
# Por ahora, esta línea se comentará o causará un NameError si se descomenta.
# configurar_red(vm_id, parametros_red, session=session, client=client)

    registrar_evento(f"Complemento finalizado correctamente. VM generada: {vm_id}")

# EJECUCIÓN DEL COMPLEMENTO
if __name__ == "__main__":
  ejecutar_complemento()

Inicio de ejecución del complemento...
[SIMULACIÓN] Conexión ficticia a http://127.0.0.1:2633/RPC2 como oneadmin
[SIMULACIÓN] Creación de VM 'vm_ctf_local_01' desde plantilla 1
[LOG] [2026-01-08T01:21:22] Complemento finalizado correctamente. VM generada: sim-vm_ctf_local_01


In [ ]:
ejecutar_complemento()

Inicio de ejecución del complemento...
[SIMULACIÓN] Conexión ficticia a http://127.0.0.1:2633/RPC2 como oneadmin
[SIMULACIÓN] Creación de VM 'vm_ctf_local_01' desde plantilla 1
[LOG] [2026-01-08T01:21:22] Complemento finalizado correctamente. VM generada: sim-vm_ctf_local_01
